Lecture 9-10 will cover iterators, operator overloading, and exceptions.

Reference
 * [2] Section 9.8-9.10

# A short note on computer assignments

* While not strictly enforced by Python, standard naming conventions should be followed.
  * If I write with camel-case `StandardDeck`, it means this is a class.
  * If I write `like_this`, it is a variable/function/method.
  * Use PyCharm's refactor->rename functionality for fixing this!

# Error handling

When dealing with user input, errors are almost inevitable. It's hard to beforehand know if something will raise an exception (i.e. cause an error)

The way to deal with this is to *try* to execute segments, and deal with the errors if they occur:

In [1]:
for x in range(2,-1,-1):
    try:
        print(1.0 / x)
    except:
        print("inf")

0.5
1.0
inf


Though, we probably want to check *what* error occured, and deal with each specifically

In [2]:
for x in range(2,-1,-1):
    try:
        print(1.0 / y) # Opps, wrong variable, not defined
    except:
        print("inf") # Not correct for this error

inf
inf
inf


In [3]:
for x in range(2,-1,-1):
    try:
        print(1.0 / x) # Opps, wrong variable, not defined
    except ZeroDivisionError:
        print("NaN") # Not correct for this error

0.5
1.0
NaN


In [35]:
for x in range(2,-1,-1):
    try:
        print(1.0 / y) # Opps, wrong variable again
    except ZeroDivisionError:
        print("inf")
    except NameError:
        print("Wrong symbol!")

Wrong symbol!
Wrong symbol!
Wrong symbol!


### Using "as" to obtain info from errors

In [2]:
import sys

try:
    f = open('myfile.txt')
    s = f.readline()
    i = int(s.strip())
except OSError as err:
    print("OS error: {0}".format(err))
except ValueError:
    print("Could not convert data to an integer.")
except:
    print("Unexpected error:", sys.exc_info()[0])
    # We re-raise the last exception:
    raise

OS error: [Errno 2] No such file or directory: 'myfile.txt'


### Finally and else

In [14]:
def divide(x, y):
    try:
        result = x / y
    except ZeroDivisionError:
        print("division by zero!")
    else:
        # Will run if we don't have an exception first:
        print("result is", result)
    finally:
        # This will always be executed last
        print("executing finally clause")

In [11]:
divide(0,0)

division by zero!
executing finally clause


In [13]:
divide(2,3)

result is 0.6666666666666666
executing finally clause


# Making your own errors

An exception is just a class:

In [28]:
class NegativeValueError(Exception):
    def __init__(self, value):
        self.value = value
    def __str__(self):
        return "Value is " + repr(self.value)

Catching the exception you can get any stored values like any object:

In [29]:
def my_function(x):
    if x < 0:
        raise NegativeValueError(x)
    return x * 3

In [30]:
x = -3
try:
    my_function(x)
except NegativeValueError as e:
    my_function(abs(x))

Uncaught exceptions prints the string representations of the class:

In [31]:
my_function(-2)

NegativeValueError: Value is -2

## Possible custom exceptions

** Question **
* In your Poker-game library, what would be some suitable exceptions to implement?

...

...

...

...

...

...

* MissingCardError
* OutOfMoneyError
* EmptyDeckError

# Operator overloading special methods

You have seen a few overloaded operators, but there are several important methods.

They are all named according to `__name__` convention (two underscores around the name).

Some (possibly useful) unary operators:
* repr: `repr(x)` (data **repr**esentation)
* len: `len(x)`
* abs: `abs(x)`
* neg: `-a`

and many other things


Common conversions:
* str
* int
* float
* complex
* bool

Lets test out a bunch of these:

In [3]:
class MyClass:
    def __init__(self, x):
        self.x = x

    def __bool__(self):
        return self.x > 0

    def __len__(self):
        return self.x

    def __neg__(self):
        return MyClass(-self.x)

    def __str__(self):
        return "x = " + str(self.x)
    
    def __repr__(self):
        return "MyClass(" + repr(self.x) + ")"

In [7]:
x = MyClass(3)
if x:
    print(x, "has length", len(x))
print(repr(-x))

x = 3 has length 3
MyClass(-3)


### Binary operators

There are a massive amount of binary operators one can overload: `+ * - / // | & << **`.

If is often in the case that both sides of the infix operator has the same type, and the output from the computation is also of the same type, but this is not strictly necessary.

Python will call the suitable special method by doing `a + b === a.__add__(b)`, with similar names for the multitude of other operators (`mul`, `div`, etc.), If `a.__add__` isn't found, then the corresponding "reverse" operation is used: `b.__radd__(a)` (if available).

Many operators have extended assignments as well, and these can be accessed through special methods as well, e.g. `a += b === a.__iadd__(b)`.

In [11]:
class Fraction:
    def __init__(self, num: int, denom: int):
        from math import gcd
        x = gcd(num, denom)
        self.num = num // x
        self.denom = denom // x

    def __str__(self):
        return str(self.num) + "/" + str(self.denom)
    
    def __float__(self):
        return self.num / self.denom

    def __add__(self, other: Fraction):
        # Expecting "other" to be a Fraction.
        # The result is a Fraction as well.
        return Fraction(self.num * other.denom + other.num * self.denom,
                        self.denom * other.denom)
    
    def __iadd__(self, other: Fraction):
        # self += other
        # We want to modify "self"!
        self.num = self.num * other.denom + other.num * self.denom
        self.denom *= other.denom
        return self # (optional behavior)
        
    def __mul__(self, other: int):
        return Fraction(self.num * other.num, self.denom * other.denom)

    def __pow__(self, exponent: int):
        # This assumes that the exponent is an integer
        return Fraction(self.num ** exponent, self.denom ** exponent)
    

In [12]:
x = Fraction(30, 40)
y = Fraction(2, 3)

q = y ** 2
x += q
z = x + y
w = x * y

print(x)
print(float(z))
print(q)
print(w)

43/36
1.8611111111111112
4/9
43/54


** Questions **
* In Python, will `a * b` always give the same result as `b * a`? 
* How about `a + b` and `b + a`?
* Will `a += b` always be the same as `a = a + b`?

### Comparison operators

Comparison operators have a few special rules, because they have no "reversed" operations.
Instead, `a < b` is used like `b > a` when `>`-operation is missing.

Implementing at least one comparison operator from `__lt__`, `__gt__` or `__cmp__` is necessary for comparisons (sort will try them in that order). Typically, `__lt__` is implemented.

In [13]:
class MyTestClass:
    """ A test class where only one instance 
    variable determines the comparison operators
    """
    
    def __init__(self, val, val2):
        self.val = val
        self.val2 = val2

    def __eq__(self, other):
        return abs(self.val) == abs(other.val)

    def __lt__(self, other): # We only check the magnitude:
        return abs(self.val) < abs(other.val)

In [14]:
x = MyTestClass(-34, "Hello")
y = MyTestClass(34, "World")
print(x > y) # Python will look for a match, and will call y < x (which should be mathematically equivalent)
print(x == y)

False
True


If `__eq__` is not implemented, the `is` operator is used instead.

In [15]:
class MyTestClass:
    def __init__(self, val):
        self.val = val

    def __lt__(self, other): # We only check the magnitude:
        return abs(self.val) < abs(other.val)

In [16]:
x = MyTestClass(23)
y = MyTestClass(23)
x == y

False

This strange default behavior is rarely what we want! Therefore, `__eq__` is recommended to overload if `__lt__` has been overloaded.

## Indexing overloading

In [17]:
class LoggingList(list):
    def __setitem__(self, pos, val):
        print("Setting items at:", pos)
        super().__setitem__(pos, val)
        
    def __getitem__(self, pos):
        print("Getting items at:", pos)
        return super().__getitem__(pos)

    def __delitem__(self, pos):
        print("Deleting items at:", pos)
        return super().__delitem__(pos)

In [18]:
x = LoggingList()
x += [1,2,3,4]

x[0:2] = [123, 456]
y = x[2:]
del x[0]

Setting items at: slice(0, 2, None)
Getting items at: slice(2, None, None)
Deleting items at: 0


## Comment on methods vs operators

This kind of compact notation is often sought by programmers, why have
```python
x.add(y)
```
when you could have
```python
x + y
```

But it's often the case that we are not working with mathematics, but strings, lists, graphical objects, and other types of custom objects.
Therefore, it's often not clear what the operator will actually do (languages like Java disallow this operator overloading for this reason).

When comparing two Car-objects, do we just check if they are of the same model? Same color? Model and color? Same registration number? Top speed?

Operators should not be overused, since it is often more clear to write a descriptive method name. Use it only when the purpose is clear.

# Iterators and generators

In the course so far, we've used multiple examples of the for-loop:

In [1]:
for i in range(5):
    print(i, end=",")

0,1,2,3,4,

In [2]:
for x in "Hello World":
    print(x, end=", ")

H, e, l, l, o,  , W, o, r, l, d, 

In [3]:
for x in zip([6,3,4], "ABC"):
    print(x, end=", ")

(6, 'A'), (3, 'B'), (4, 'C'), 

All these examples are very different. The `range` keeps generating new numbers until the max value is reached. In the string, we go letter by letter until the end of the string.
The `for`-loop is compact, convenient, and easy to read.

### The `for`-loop

Lets assume we have a loop like:
```python
for a in b:
    ...
```
When the `for`-loop starts, it asks for
```python
it = iter(b)    # Looks for b.__iter__()
```
and then it asks for
```python
a = next(it)    # Looks for it.__next__()
```
until a `StopIteration`-exception is raised.

Lets try it out, step-by-step

In [4]:
b = "Hello"
it = iter(b)
print(it)

In [5]:
print( next(it) )
print( next(it) )
print( next(it) )
print( next(it) )
print( next(it) )

H
e
l
l
o


So far so good, what happens when we ask one more time?

In [6]:
next(it)

StopIteration: 

**To sum it up:** 
1. The object needs support the function `iter`
2. Whatever it returned from `iter` must support the function `next`

### Lets create a custom iterator

In [19]:
class CollatzGenerator:
    def __init__(self, n):
        self.n = n

    def __iter__(self):
        # We'll reuse the class generator itself 
        # as the iterator for this simple case
        return self
    
    def __next__(self):
        if self.n == 1:
            raise StopIteration
        if self.n % 2 == 0:
            self.n //= 2
        else:
            self.n = self.n*3 + 1
        
        return self.n

In [20]:
# This should produce 3, 10, 5, 16, 8, 4, 2, 1.
x = CollatzGenerator(6)
for n in x:
    print(n, end=", ")

3, 10, 5, 16, 8, 4, 2, 1, 

Note! In this `CollatzGenerator`, the iterator is the same as the generator itself.
If we try to use `x` again, it will already have reached 1, as the (modified) iterator *is* the generator.

In [21]:
print( x.n )

1


### Another example, the infinite loop!

Lets create a loop over all natural numbers. We would like to have
```python
nat = NaturalNumbers():
for x in nat:
   ...
```
Of course, it's not very good that we modified the starting point of the `nat` (in case we want to reuse it).

We should therefor avoid `def __iter__(self): return self`, and instead return a different iterator.

In [19]:
class NaturalNumbers:
    def __iter__(self):
        # Lets declare an iterator internally:
        class NaturalNumbers_It:
            def __init__(self):
                self.n = 0
       
            def __next__(self):
                self.n += 1
                return self.n
        # Each time we start a for-loop, 
        # we'll return a brand new iterator:
        return NaturalNumbers_It()

In [20]:
nat = NaturalNumbers()

In [23]:
tot = 0
for x in nat:
    tot += x
    # We might have some condition to stop that 
    # doesn't depend on only one the value of x!
    if tot > 1000:
        break
print(tot, x)

1035 45


## Generators

The interested student can also look up the `yield` statement. For example here:
http://www.python-course.eu/python3_generators.php

The for-loop will obtain each `yield`'ed value, and the function will be frozen until the next yielded value is requested:

In [451]:
def countfromto(a, b):
    while a < b:
        print("Yielding a new value!")
        yield a
        a += 1

for i in countfromto(10, 15):
    print(i)

Yielding a new value!
10
Yielding a new value!
11
Yielding a new value!
12
Yielding a new value!
13
Yielding a new value!
14


# Enums

Often, we want to store a simple state, for example, whether a game is
* Not started
* Started
* Ended
* Paused
and we want to quickly have logic that can check for these states:

```python
if game.status() == "not started":
    game.start()
```

But comparing with strings is a bit slow (if the use case is more performance critical), and also introduces the possibilities of having a typo. It also can't be automatically refactored. It's generally _very_ frowned upon to keep hard-coded values (including strings) spread out throughout the code as well.

The solution? Enums.

In [23]:
import enum

class GameState(enum.Enum):
    not_started = 0
    started = 1
    ended = 2
    paused = 3

These variables can be cheaply compared, with the extra safety of typing, completion, etc.:

In [24]:
for g in GameState: # We can loop over them!
    print(g)
# And compare them
print(GameState.paused == GameState.paused)
print(GameState.paused != GameState.ended)

GameState.not_started
GameState.started
GameState.ended
GameState.paused
True
True


We can also create IntEnum classes

In [25]:
class GameState2(enum.IntEnum):
    not_started = 0
    started = 1
    ended = 2
    paused = 3
    
    def __str__(self):
        # Added a custom string conversion! 
        # (the default will print GameState2.*****)
        return 'abcd'[self.value]

Here I also demonstrate the custom string conversion possible, using the value. By default, the `self.name` parameter is used.

In [26]:
print(GameState2.started)

b


Which lets us convert values to integers

In [27]:
GameState2.started + 2

3

Such as comparing them:

In [28]:
print(GameState2.paused < GameState2.ended)

False


which is explictly disallowed with normal Enums (a type safety thing)

In [30]:
print(GameState.paused < GameState.ended)

TypeError: '<' not supported between instances of 'GameState' and 'GameState'

You can still overload the '<' operator if you want.

There is a handy decorator which ensures uniqueness of the values in the enum (often the case):

In [31]:
@enum.unique
class GameState2(enum.IntEnum):
    not_started = 0
    started = 1
    ended = 2
    paused = 2 # Opps

ValueError: duplicate values found in <enum 'GameState2'>: paused -> ended